In [ ]:
!pip install -r requirements.txt

In [83]:
from dotenv import load_dotenv
load_dotenv()

True

In [88]:

from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from openai import max_retries
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
# doesn't work with o1-mini and deepseek-r1.
# o1 was unable to format input for the calculator tool

# llm = ChatOllama(
#     # model="deepseek-r1",
#     model="llama3.2",
#     temperature=0,
#     max_retries=-1
#     # other params...
# )

In [89]:

response = llm.invoke("what is (4.5*2.1)^2.2?")
print(response)


content='To calculate \\((4.5 \\times 2.1)^{2.2}\\), we first need to compute \\(4.5 \\times 2.1\\):\n\n\\[\n4.5 \\times 2.1 = 9.45\n\\]\n\nNext, we raise \\(9.45\\) to the power of \\(2.2\\):\n\n\\[\n9.45^{2.2}\n\\]\n\nUsing a calculator, we find:\n\n\\[\n9.45^{2.2} \\approx 103.67\n\\]\n\nTherefore, \\((4.5 \\times 2.1)^{2.2} \\approx 103.67\\).' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 22, 'total_tokens': 164, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None} id='run-070d7fe6-4ff5-41aa-8134-abdfdb252a17-0' usage_metadata={'input_tokens': 22, 'output_tokens': 142, 'total_tokens': 164, 'input_token_details': {'audio': 0, 'cache_read': 0}, '

In [90]:
from langchain.chains import LLMMathChain # Replace with your actual LLM class
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [91]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [92]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [93]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [94]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
To solve the expression \((4.5 \times 2.1)^{2.2}\), I first need to calculate the product \(4.5 \times 2.1\) and then raise the result to the power of \(2.2\).

Action: Calculator
Action Input: 4.5 * 2.1

Observation: Answer: 9.450000000000001
Thought:Now that I have the product \(4.5 \times 2.1 = 9.45\), I need to raise this result to the power of \(2.2\).

Action: Calculator  
Action Input: 9.45 ** 2.2

Observation: Answer: 139.9426129833306
Thought:I now know the final answer.  
Final Answer: 139.9426129833306

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.9426129833306'}